## This is a test of training a custom NER model using Spacy and NBA articles.
###### It did not go too well, but it was a great start on figuring out how to train Custom models on Spacy using their formats and all

In [14]:
# import spacy
import random
import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

In [15]:
TRAIN_DATA = [('The Lakers are on the brink of elimination in the Western Conference playoffs', {'entities': [(50, 68, 'Conference'), (4, 10, 'Team')]}), ('Fortunately for Los Angeles, LeBron James has a favorable track record in Game 6s', {'entities': [(29, 41, 'Person'), (16, 27, 'Team')]}), ('The Lakers will put that record to the test Thursday at 10:30 p.m. against the Suns.', {'entities': [(83, 79, 'Team'), (44, 65, 'Time'), (4, 10, 'Team')]}), ('How good is James in the sixth game of a series?', {'entities': [(17, 12, 'Person')]}), ("There was almost no way to stop Damian Lillard in Tuesday night's Trail Blazers vs. Nuggets game, and Austin Rivers bore witness to that.", {'entities': [(102, 115, 'Person'), (84, 91, 'Team'), (66, 79, 'Team'), (50, 65, 'Time'), (32, 46, 'Person')]}), ('Rivers was tasked with guarding Lillard for the majority of the game. Rivers was mostly helpless in guarding the All-Star but still found the experience really fun.', {'entities': [(70, 76, 'Person'), (32, 39, 'Person'), (0, 5, 'Person')]}), ('Lillard went 17-of-24 from�\xa0the field�\xa0and made 12 3-point shots on 17 attempts.', {'entities': [(41, 77, 'Stat'), (13, 21, 'Stat'), (7, 0, 'Person')]}), ('Not only did he set the record for most playoff 3-pointers�\xa0made in a single game, but he tied the seventh-highest single-game scoring output by a player in NBA playoff history with 55 points.', {'entities': [(181, 190, 'Stat')]}), ("However, even despite Lillard's heroics, the Trail Blazers weren't able to emerge with the win.", {'entities': [(45, 58, 'Team'), (22, 31, 'Person')]}), ('The Nuggets took home a 147-140 victory in double-overtime.', {'entities': [(24, 31, 'Stat'), (4, 11, 'Team')]}), ("Maybe that's why Rivers thought guarding Lillard was fun last night.", {'entities': [(57, 67, 'Time'), (41, 48, 'Person'), (17, 23, 'Person')]}), ("Had the Blazers emerged with the win, perhaps he'd have a bit of a different take on having to guard a nearly unstoppable player.", {'entities': [(8, 15, 'Team')]}), ('One Lakers legend had no problem voicing his disappointment and frustration after watching Phoenix brutally beat down Los Angeles in a 115-85 victory.', {'entities': [(135, 141, 'Stat'), (118, 129, 'Team'), (91, 98, 'Team'), (4, 10, 'Team')]}), ('During Tuesday\'s�\xa0"NBA on TNT" postgame show, Shaquille O\'Neal blasted�\xa0the Lakers, a team that suddenly finds itself on the brink of elimination heading back to the Staples Center for�\xa0Game 6.', {'entities': [(74, 80, 'Team'), (45, 61, 'Person'), (7, 16, 'Time')]}), ('"As a former Laker, I have G-14 classification to say what I\'m about to say," O\'Neal told the rest of the panel.', {'entities': [(78, 84, 'Person'), (13, 18, 'Team')]}), ("While James totaled 24 points, seven assists�\xa0and five rebounds in the loss, he�\xa0didn't play up to his usual standards.", {'entities': [(49, 62, 'Stat'), (31, 44, 'Stat'), (20, 29, 'Stat'), (6, 11, 'Person')]}), ('But now these Eastern Conference powerhouses are heading into the second round with the full set of stars ready to go.', {'entities': [(14, 32, 'Conference')]}), ('The Nets never had their full "Big Three" available against the Bucks — Kyrie Irving missed the first game, and James Harden sat out the other two — so it would be unwise to look into the results too deeply.', {'entities': [(112, 124, 'Person'), (72, 84, 'Person'), (0, 8, 'Team')]}), ('Six different Milwaukee players averaged at least 10 points per game in that first-round sweep, including reigning MVP Giannis Antetokounmpo, who posted 23.5 points, 15.0 rebounds and 7.8 assists against Miami.', {'entities': [(204, 209, 'Team'), (184, 195, 'Stat'), (166, 179, 'Stat'), (153, 164, 'Stat'), (119, 140, 'Person'), (50, 68, 'Stat'), (14, 23, 'Team')]}), ('The Nets, meanwhile, eliminated a battered Celtics squad in five games, only dropping Game 3 in Boston.', {'entities': [(96, 102, 'Team'), (43, 50, 'Team'), (8, 0, 'Team')]}), ('Butler averaged just 14.5 points per game while shooting less than 30 percent from the field.', {'entities': [(67, 92, 'Stat'), (21, 41, 'Stat'), (0, 6, 'Person')]})]

In [4]:
TRAIN_DATA

[('The Lakers are on the brink of elimination in the Western Conference playoffs',
  {'entities': [(50, 68, 'Conference'), (4, 10, 'Team')]}),
 ('Fortunately for Los Angeles, LeBron James has a favorable track record in Game 6s',
  {'entities': [(29, 41, 'Person'), (16, 27, 'Team')]}),
 ('The Lakers will put that record to the test Thursday at 10:30 p.m. against the Suns.',
  {'entities': [(83, 79, 'Team'), (44, 65, 'Time'), (4, 10, 'Team')]}),
 ('How good is James in the sixth game of a series?',
  {'entities': [(17, 12, 'Person')]}),
 ("There was almost no way to stop Damian Lillard in Tuesday night's Trail Blazers vs. Nuggets game, and Austin Rivers bore witness to that.",
  {'entities': [(102, 115, 'Person'),
    (84, 91, 'Team'),
    (66, 79, 'Team'),
    (50, 65, 'Time'),
    (32, 46, 'Person')]}),
 ('Rivers was tasked with guarding Lillard for the majority of the game. Rivers was mostly helpless in guarding the All-Star but still found the experience really fun.',
  {'entities':

In [16]:
# def train_spacy(data,n_iter):
#     TRAIN_DATA = data
#     nlp = spacy.blank('en')
#     if 'ner' not in nlp.pipe_names:
#         ner = nlp.create_pipe('ner')
#         nlp.add_pipe('ner', last=True)
    
#     for _, annotations in TRAIN_DATA:
#         for ent in annotations.get('entities'):
#             ner.add_label(ent[2])

#     other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
#     with nlp.disable_pipes(*other_pipes):  # only train NER
#         optimizer = nlp.begin_training()
#         for itn in range(n_iter):
#             random.shuffle(TRAIN_DATA)
#             losses = {}
#             for text, annotations in TRAIN_DATA:
#                 nlp.update(
#                     [text],  
#                     [annotations],  
#                     drop=0.3,  
#                     sgd=optimizer,
#                     losses=losses)
#             print(losses)
#     return nlp
nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

for text, annot in tqdm(TRAIN_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./train.spacy") # save the docbin object

100%|█████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 804.70it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [19]:
!python -m spacy init config --lang en --pipeline ner config.cfg --force

[!] To generate a more effective transformer-based config (GPU-only), install
the spacy-transformers package and re-run this command. The config generated now
does not use transformers.
[i] Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [20]:
TEST_DATA = [('After witnessing his basketball masterpiece spread over four quarters and two overtime periods and too many head-shaking big buckets to count, everyone went home Tuesday with this about Damian Lillard.', {'entities': [(186, 200, 'Person'), (162, 169, 'Time')]}), ('And he’s two elimination games, if he’s lucky, of going away for the summer again after Denver’s�\xa0147-140 2OT victory�\xa0in Game 5.', {'entities': [(97, 104, 'Stat'), (96, 88, 'Team')]}), ('Lillard was truly brilliant for the Portland Trail Blazers when it counted in Game 5 against the Nuggets and there was nothing Denver could do.', {'entities': [(127, 133, 'Team'), (97, 104, 'Team'), (36, 58, 'Team'), (0, 7, 'Person')]}), ('The Clippers, despite getting on the board with a Game 3 victory over the Mavericks, have struggled to contain Dallas superstar Luka Doncic.', {'entities': [(128, 139, 'Person'), (111, 117, 'Team'), (74, 83, 'Team'), (12, 0, 'Team')]}), ('The Slovenian wunderkind has averaged 38.0 points, 8.7 rebounds and 9.0 assists, with 51.9%/45.9%/48.1% shooting splits.', {'entities': [(86, 112, 'Stat'), (68, 79, 'Stat'), (51, 63, 'Stat'), (38, 49, 'Stat')]}), ('Coach Rick Carlisle sounded an optimistic note in his off-day media availability, telling reporters, Questionable generally, from what I’ve seen this year with other teams and our team, is more likely to play than not play in most cases.', {'entities': [(0, 19, 'Person')]}), ('Zamalek (Egypt) today defeated US Monastir (Tunisia) 76-63 to win the inaugural Basketball Africa League (BAL) Championship, which took place at the Kigali Arena in Rwanda and was broadcast to fans in 215 countries and territories in 15 languages.', {'entities': [(53, 58, 'Stat'), (31, 42, 'Team'), (7, 0, 'Team')]}), ('Down 0-1 in the series and trailing by 13 at the half (57-44) in Game 2, the Knicks rallied in the second half, outscoring the Hawks 57-35 in front of a raucous Madison Square Garden crowd to�\xa0even the series�\xa0at 1-1.', {'entities': [(211, 214, 'Stat'), (133, 138, 'Stat'), (127, 132, 'Team'), (77, 83, 'Team'), (55, 60, 'Stat'), (27, 41, 'Stat'), (5, 8, 'Stat')]}), ('Derrick Rose was inserted into the starting lineup for the�\xa0second half�\xa0and responded with 12 points, four rebounds and three assists in the second half, and finished with a team-high 26 points – his�\xa0first 25+ point playoff game�\xa0since 2015 against the Cavs.', {'entities': [(251, 255, 'Team'), (234, 238, 'Time'), (183, 192, 'Stat'), (119, 132, 'Stat'), (101, 114, 'Stat'), (90, 99, 'Stat'), (0, 12, 'Person')]}), ('Between the regular season and playoffs, the Nets are 5-0 against the Celtics this season, holding the Celtics to an average of 101.8 points per game.', {'entities': [(128, 149, 'Stat'), (103, 110, 'Team'), (70, 77, 'Team'), (54, 57, 'Stat'), (41, 49, 'Team')]}), ('Through the first two games of the series, Tim Hardaway Jr. has been on fire,�\xa0averaging 24.5 points�\xa0on 63% shooting form the field and 64.7% from 3-point range.', {'entities': [(135, 159, 'Stat'), (103, 115, 'Stat'), (88, 99, 'Stat'), (43, 59, 'Person')]}), ('Kawhi Leonard is averaging a playoff-high 23.5 points in the�\xa0first half, while shooting 62.5% from the filed, 66.7% from 3-point range and 92.9% from the free throw line.', {'entities': [(139, 170, 'Stat'), (110, 134, 'Stat'), (88, 108, 'Stat'), (42, 53, 'Stat'), (0, 13, 'Person')]}), ('The Brooklyn Nets took care of business on Tuesday night, finishing off�\xa0their first-round series�\xa0against the Boston Celtics with�\xa0a 123-109 victory�\xa0in Game 5.', {'entities': [(131, 138, 'Stat'), (109, 123, 'Team'), (43, 56, 'Time'), (4, 17, 'Team')]})]

In [21]:
for text, annot in tqdm(TEST_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./test.spacy") # save the docbin object

100%|█████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 238.68it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [22]:
!python -m spacy train config.cfg --output /training/ --paths.train train.spacy --paths.dev test.spacy --training.eval_frequency 10 --training.max_steps 100 --gpu-id -1

[+] Created output directory: \training
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     41.95    0.00    0.00    0.00    0.00
  1      10          2.04    654.12    0.00    0.00    0.00    0.00
  3      20          4.60    160.50    0.00    0.00    0.00    0.00
  4      30          5.68    142.67    3.67   22.22    2.00    0.04
  6      40          1.41    108.72   15.49   26.19   11.00    0.15
  8      50          0.93     72.34   19.51   25.00   16.00    0.20
  9      60         34.82    124.36   30.33   28.83   32.00    0.30
 11      70          3.10     98.32   37.43   40.23   35.00    0.37
 13      80         

[2021-06-02 23:31:07,287] [INFO] Set up nlp object from config
[2021-06-02 23:31:07,298] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-06-02 23:31:07,302] [INFO] Created vocabulary
[2021-06-02 23:31:07,302] [INFO] Finished initializing nlp object
[2021-06-02 23:31:07,462] [INFO] Initialized pipeline components: ['tok2vec', 'ner']



 16     100          1.34     41.33   58.16   59.38   57.00    0.58
[+] Saved pipeline to output directory
\training\model-last


In [23]:
nlp1 = spacy.load(R"/training/model-best")

In [24]:
doc = nlp1("Luka Doncic scored 32 points against the Los Angeles Clippers")

In [ ]:
spacy.displacy.render(doc, style='ent',jupyter = True)